# 04차시: Pandas로 금융 데이터 다루기 (읽기, 쓰기, 선택, 필터링)

## 학습 목표
- Python 금융 라이브러리를 사용하여 한국 주가 데이터 가져오기
- CSV, Excel 파일로 데이터 저장하고 불러오기
- 특정 날짜, 기간의 데이터 선택하기
- 조건에 맞는 데이터 필터링하기

## 학습 내용
1. FinanceDataReade로 한국 주가 데이터 가져오기
2. CSV 파일 읽기/쓰기
3. Excel 파일 읽기/쓰기
4. 날짜/기간 데이터 선택
5. 조건 필터링
6. 종합 실습

---
실제 한국 주가 데이터를 활용하여 Pandas의 핵심 기능을 실습합니다.


## 1. FinanceDataReader로 한국 주가 데이터 가져오기

### FinanceDataReader란?
- 주식·지수·환율·암호화폐 같은 금융 시계열 데이터를 파이썬에서 아주 쉽게 불러오게 해주는 오픈소스 라이브러리
- KOSPI / KOSDAQ 종목, 미국 주식 (NASDAQ, NYSE) 가격 정보 제공
- 환율, 암호화폐 정보 제공

### 주요 함수
```python
import FinanceDataReader as fdr

# 일별 OHLCV 조회
fdr.DataReader(ticker, fromdate, todate)

# 종목 코드로 종목명 조회
stock_list = fdr.StockListing('KRX')
stock_list[stock_list['Code'] == ticker]['Name'].values[0]
```
- `fromdate`: 시작 날짜 (예: '20220101')
- `todate`: 종료 날짜 (예: '20241231')
- `ticker`: 종목 코드 (예: '005930')


In [1]:
# FinanceDataReader 설치
%pip install finance-datareader openpyxl -q

In [2]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np

In [ ]:
df = fdr.StockListing('KRX')[['Code', 'Name']]
df.head()

,Code,Name
0,005930,삼성전자
1,000660,SK하이닉스
2,373220,LG에너지솔루션
3,005935,삼성전자우
4,207940,삼성바이오로직스


In [4]:
STOCK_NAMES = dict(zip(df['Code'], df['Name']))

In [5]:
# 삼성전자(005930) 주가 데이터 다운로드
print("[삼성전자 주가 데이터 다운로드]")
print("=" * 60)

# 종목 정보
ticker = "005930"
stock_name = STOCK_NAMES.get(ticker, ticker)  # 딕셔너리에 없으면 종목코드 그대로 사용
print(f"종목코드: {ticker}")
print(f"종목명: {stock_name}")

# 주가 데이터 조회 (2022년 1월 1일부터 현재까지)
# FDR은 'YYYY-MM-DD' 형식 또는 datetime 객체 사용
df = fdr.DataReader(ticker, "2022-01-01", "2024-12-31")

print(f"\n데이터 기간: {df.index[0].strftime('%Y-%m-%d')} ~ {df.index[-1].strftime('%Y-%m-%d')}")
print(f"데이터 크기: {df.shape[0]}행 x {df.shape[1]}열")
print(f"\n열 이름: {df.columns.tolist()}")

df.head()

[삼성전자 주가 데이터 다운로드]
종목코드: 005930
종목명: 삼성전자

데이터 기간: 2022-01-03 ~ 2024-12-30
데이터 크기: 735행 x 6열

열 이름: ['Open', 'High', 'Low', 'Close', 'Volume', 'Change']


,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-01-03,79400,79800,78200,78600,13502112,0.003831
2022-01-04,78800,79200,78300,78700,12427416,0.001272
2022-01-05,78800,79000,76400,77400,25470640,-0.016518
2022-01-06,76700,77600,76600,76900,12931954,-0.006460
2022-01-07,78100,78400,77400,78300,15163757,0.018205


## 2. CSV 파일 읽기/쓰기

### CSV란?
- Comma Separated Values (쉼표로 구분된 값)
- 가장 널리 사용되는 데이터 저장 형식
- 텍스트 기반이라 용량이 작고 호환성이 높음

### 주요 함수
| 함수 | 설명 |
|------|------|
| `df.to_csv('파일명.csv')` | DataFrame을 CSV로 저장 |
| `pd.read_csv('파일명.csv')` | CSV를 DataFrame으로 읽기 |

### 주요 옵션
- `index=False`: 인덱스 저장 안함
- `index_col=0`: 첫 번째 열을 인덱스로 사용
- `parse_dates=True`: 날짜 자동 변환
- `encoding='utf-8'`: 인코딩 지정


In [6]:
# CSV 파일로 저장
print("[CSV 파일 저장]")
print("=" * 60)

# 기본 저장
df.to_csv('삼성전자_stock.csv')
print("저장 완료: 삼성전자_stock.csv")

# 저장된 파일 확인
!ls -la 삼성전자_stock.csv

# CSV 파일 내용 미리보기 (처음 5줄)
print("\n[CSV 파일 내용 미리보기]")
!head -5 삼성전자_stock.csv

[CSV 파일 저장]
저장 완료: 삼성전자_stock.csv
-rw-r--r-- 1 root root 47564 Jan  7 03:08 삼성전자_stock.csv

[CSV 파일 내용 미리보기]
Date,Open,High,Low,Close,Volume,Change
2022-01-03,79400,79800,78200,78600,13502112,0.003831417624521105
2022-01-04,78800,79200,78300,78700,12427416,0.0012722646310432406
2022-01-05,78800,79000,76400,77400,25470640,-0.016518424396442133
2022-01-06,76700,77600,76600,76900,12931954,-0.006459948320413411


In [7]:
# CSV 파일 불러오기
print("[CSV 파일 불러오기]")
print("=" * 60)

# 기본 읽기
df_csv = pd.read_csv('삼성전자_stock.csv')
print("기본 읽기 결과:")
print(df_csv.head(3))
print(f"\n인덱스 타입: {type(df_csv.index)}")

print("\n" + "=" * 60)
# 날짜를 인덱스로 설정하여 읽기
df_csv = pd.read_csv('삼성전자_stock.csv', index_col=0, parse_dates=True)

df_csv.head()

[CSV 파일 불러오기]
기본 읽기 결과:
         Date   Open   High    Low  Close    Volume    Change
0  2022-01-03  79400  79800  78200  78600  13502112  0.003831
1  2022-01-04  78800  79200  78300  78700  12427416  0.001272
2  2022-01-05  78800  79000  76400  77400  25470640 -0.016518

인덱스 타입: <class 'pandas.core.indexes.range.RangeIndex'>



,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-01-03,79400,79800,78200,78600,13502112,0.003831
2022-01-04,78800,79200,78300,78700,12427416,0.001272
2022-01-05,78800,79000,76400,77400,25470640,-0.016518
2022-01-06,76700,77600,76600,76900,12931954,-0.006460
2022-01-07,78100,78400,77400,78300,15163757,0.018205


## 3. Excel 파일 읽기/쓰기

### Excel 파일 장점
- 시트(Sheet) 기능으로 여러 데이터 관리 가능
- 서식, 차트 등 풍부한 기능
- 비개발자와의 협업에 용이

### 주요 함수
| 함수 | 설명 |
|------|------|
| `df.to_excel('파일명.xlsx')` | DataFrame을 Excel로 저장 |
| `pd.read_excel('파일명.xlsx')` | Excel을 DataFrame으로 읽기 |

### 주의사항
- `openpyxl` 라이브러리 필요
- CSV보다 파일 크기가 큼


In [8]:
# openpyxl 설치 (Excel 파일 처리용)
# %pip install openpyxl -q

# Excel 파일로 저장
print("[Excel 파일 저장]")
print("=" * 60)

df.to_excel('삼성전자_stock.xlsx', sheet_name='주가데이터')
print("저장 완료: 삼성전자_stock.xlsx")

# 저장된 파일 확인
!ls -la 삼성전자_stock.xlsx

[Excel 파일 저장]
저장 완료: 삼성전자_stock.xlsx
-rw-r--r-- 1 root root 40587 Jan  7 03:08 삼성전자_stock.xlsx


In [9]:
# Excel 파일 불러오기
print("[Excel 파일 불러오기]")
print("=" * 60)

df_excel = pd.read_excel('삼성전자_stock.xlsx', index_col=0)
df_excel.head(3)

[Excel 파일 불러오기]


,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-01-03,79400,79800,78200,78600,13502112,0.003831
2022-01-04,78800,79200,78300,78700,12427416,0.001272
2022-01-05,78800,79000,76400,77400,25470640,-0.016518


In [10]:
print("\n" + "=" * 60)
# 여러 시트에 데이터 저장
print("[여러 시트에 데이터 저장]")

# 2022년, 2023년 데이터 분리
df_2022 = df[df.index.year == 2022]
df_2023 = df[df.index.year == 2023]

# ExcelWriter로 여러 시트 저장
with pd.ExcelWriter('삼성전자_yearly.xlsx') as writer:
    df_2022.to_excel(writer, sheet_name='2022년')
    df_2023.to_excel(writer, sheet_name='2023년')

print("저장 완료: 삼성전자_yearly.xlsx")
print(f"  - 2022년 시트: {len(df_2022)}행")
print(f"  - 2023년 시트: {len(df_2023)}행")


[여러 시트에 데이터 저장]
저장 완료: 삼성전자_yearly.xlsx
  - 2022년 시트: 246행
  - 2023년 시트: 245행


## 4. 날짜/기간 데이터 선택

### 날짜 인덱스의 장점
- 문자열로 직관적인 날짜 선택 가능
- 기간 슬라이싱 지원
- 연도, 월, 일 단위 필터링

### 주요 방법
| 방법 | 예시 |
|------|------|
| 특정 날짜 | `df.loc['2023-01-02']` |
| 기간 선택 | `df.loc['2023-01-01':'2023-06-30']` |
| 연도 필터 | `df[df.index.year == 2023]` |
| 월 필터 | `df[df.index.month == 1]` |
|요일 필터 (숫자)|	df[df.index.weekday == 0] |
|요일 필터 (이름)|	df[df.index.day_name() == 'Monday'] |

0 = Monday, 1 = Tuesday, 2 = Wednesday, 3 = Thursday, 4 = Friday, 5 = Saturday, 6 = Sunday

In [11]:
# 특정 날짜 데이터 선택
print("[특정 날짜 데이터 선택]")
print("=" * 60)

# 특정 날짜
print("2023년 1월 3일 데이터:")
print(df.loc['2023-01-03'])

print("\n" + "=" * 60)
# 기간 선택 (슬라이싱)
print("[기간 선택: 2023년 상반기]")
df_2023_h1 = df.loc['2023-01-01':'2023-06-30']
print(f"데이터 개수: {len(df_2023_h1)}개")
print("\n처음 3행:")
print(df_2023_h1.head(3))
print("\n마지막 3행:")
print(df_2023_h1.tail(3))

[특정 날짜 데이터 선택]
2023년 1월 3일 데이터:
Open      5.540000e+04
High      5.600000e+04
Low       5.450000e+04
Close     5.540000e+04
Volume    1.354703e+07
Change   -1.801802e-03
Name: 2023-01-03 00:00:00, dtype: float64

[기간 선택: 2023년 상반기]
데이터 개수: 123개

처음 3행:
             Open   High    Low  Close    Volume    Change
Date                                                      
2023-01-02  55500  56100  55200  55500  10031448  0.003617
2023-01-03  55400  56000  54500  55400  13547030 -0.001802
2023-01-04  55700  58000  55600  57800  20188071  0.043321

마지막 3행:
             Open   High    Low  Close    Volume    Change
Date                                                      
2023-06-28  72600  72700  72000  72700   8783093  0.001377
2023-06-29  73100  73400  72400  72400  12229967 -0.004127
2023-06-30  72500  72700  71700  72200  11694765 -0.002762


In [12]:
# 연도/월 기준 필터링
print("[연도/월 기준 필터링]")
print("=" * 60)

# 2023년 데이터만
df_2023 = df[df.index.year == 2023]
print(f"2023년 데이터: {len(df_2023)}개")

# 1월 데이터만 (모든 연도)
df_jan = df[df.index.month == 1]
print(f"1월 데이터 (전체 연도): {len(df_jan)}개")

# 2023년 12월 데이터
df_2023_dec = df[(df.index.year == 2023) & (df.index.month == 12)]
print(f"2023년 12월 데이터: {len(df_2023_dec)}개")
print("\n2023년 12월 데이터:")
print(df_2023_dec)

# 월요일 데이터만
print("\n" + "=" * 60)
print("[요일 기준 필터링: 월요일만]")
df_monday = df[df.index.dayofweek == 0]  # 0=월요일
print(f"월요일 데이터 개수: {len(df_monday)}개")
df_monday.head()

[연도/월 기준 필터링]
2023년 데이터: 245개
1월 데이터 (전체 연도): 62개
2023년 12월 데이터: 19개

2023년 12월 데이터:
             Open   High    Low  Close    Volume    Change
Date                                                      
2023-12-01  72400  72500  71700  72000   9871284 -0.010989
2023-12-04  72800  72900  72400  72600  10229267  0.008333
2023-12-05  72300  72400  71200  71200  12129682 -0.019284
2023-12-06  71800  72100  71600  71700   8123087  0.007022
2023-12-07  71800  71900  71100  71500   8862017 -0.002789
2023-12-08  72100  72800  71900  72600  10859463  0.015385
2023-12-11  72800  73000  72200  73000   9861960  0.005510
2023-12-12  73300  73500  73100  73500  13758646  0.006849
2023-12-13  73300  73500  72800  72800  13116766 -0.009524
2023-12-14  74100  74300  72500  73100  27567593  0.004121
2023-12-15  73800  74000  73200  73300  15419815  0.002736
2023-12-18  73300  73400  72800  72900   9690551 -0.005457
2023-12-19  73000  73400  72800  73400   8907632  0.006859
2023-12-20  74200  74900  7380

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-01-03,79400,79800,78200,78600,13502112,0.003831
2022-01-10,78100,78100,77100,78000,9947422,-0.003831
2022-01-17,77600,77800,76900,77500,8785122,0.002587
2022-01-24,75400,75800,74700,75100,13691134,-0.006614
2022-02-07,73500,73600,72400,73000,14240838,-0.013514


## 5. 조건 필터링

### 조건 필터링 기본 문법
```python
df[조건식]
```

### 비교 연산자
| 연산자 | 의미 |
|--------|------|
| `==` | 같다 |
| `!=` | 다르다 |
| `>`, `<` | 크다, 작다 |
| `>=`, `<=` | 크거나 같다, 작거나 같다 |

### 논리 연산자 (복합 조건)
| 연산자 | 의미 |
|--------|------|
| `&` | AND (그리고) |
| `\|` | OR (또는) |
| `~` | NOT (부정) |

**주의**: 각 조건은 괄호로 감싸야 합니다.


In [13]:
# 가격 조건 필터링
print("[가격 조건 필터링]")
print("=" * 60)

# 종가 70,000원 이상인 날
high_price = df[df['Close'] >= 70000]
print(f"종가 70,000원 이상: {len(high_price)}일")
print(high_price[['Close', 'Volume']].head())

print("\n" + "=" * 60)
# 종가 60,000원 이하인 날
low_price = df[df['Close'] <= 60000]
print(f"종가 60,000원 이하: {len(low_price)}일")
print(low_price[['Close', 'Volume']].head())

# 거래량 조건 필터링
print("[거래량 조건 필터링]")
print("=" * 60)

# 거래량 3천만주 이상인 날 (대량 거래일)
high_volume = df[df['Volume'] >= 30_000_000]
print(f"거래량 3천만주 이상: {len(high_volume)}일")
print("\n대량 거래일 (거래량 순 정렬):")
print(high_volume.sort_values('Volume', ascending=False)[['Close', 'Volume']].head(10))

[가격 조건 필터링]
종가 70,000원 이상: 311일
            Close    Volume
Date                       
2022-01-03  78600  13502112
2022-01-04  78700  12427416
2022-01-05  77400  25470640
2022-01-06  76900  12931954
2022-01-07  78300  15163757

종가 60,000원 이하: 156일
            Close    Volume
Date                       
2022-06-17  59800  29053450
2022-06-20  58700  34111306
2022-06-21  58500  25148109
2022-06-22  57600  23334687
2022-06-23  57400  28338608
[거래량 조건 필터링]
거래량 3천만주 이상: 39일

대량 거래일 (거래량 순 정렬):
            Close    Volume
Date                       
2024-01-11  73200  57691266
2024-08-05  71400  54608790
2024-11-13  50600  52527995
2024-03-20  76900  50106297
2024-09-19  63100  49402713
2024-11-14  49900  48510715
2024-11-18  56700  48095234
2024-08-06  72500  47295226
2024-11-15  53500  46774485
2024-07-05  87100  45791193


In [14]:
# 복합 조건 필터링
print("[복합 조건 필터링]")
print("=" * 60)

# 종가 65,000원 이상 AND 거래량 2천만주 이상
condition1 = df['Close'] >= 65000
condition2 = df['Volume'] >= 20_000_000
filtered = df[condition1 & condition2]
print(f"종가 65,000원 이상 AND 거래량 2천만주 이상: {len(filtered)}일")
print(filtered[['Close', 'Volume']].head())

print("\n" + "=" * 60)
# 상승일 찾기 (종가 > 시가)
df['상승여부'] = df['Close'] > df['Open']
up_days = df[df['상승여부'] == True]
print(f"상승일 (종가 > 시가): {len(up_days)}일")
print(f"상승률: {len(up_days) / len(df) * 100:.1f}%")

print("\n" + "=" * 60)
# 변동폭이 큰 날 (고가 - 저가 > 2,000원)
df['변동폭'] = df['High'] - df['Low']
volatile_days = df[df['변동폭'] > 2000]
print(f"변동폭 2,000원 초과일: {len(volatile_days)}일")
print(volatile_days[['Open', 'High', 'Low', 'Close', '변동폭']].head())

[복합 조건 필터링]
종가 65,000원 이상 AND 거래량 2천만주 이상: 92일
            Close    Volume
Date                       
2022-01-05  77400  25470640
2022-01-27  71300  22274777
2022-01-28  73300  21367447
2022-03-10  71200  21159248
2022-03-24  69800  37943357

상승일 (종가 > 시가): 312일
상승률: 42.4%

변동폭 2,000원 초과일: 52일
             Open   High    Low  Close   변동폭
Date                                        
2022-01-05  78800  79000  76400  77400  2600
2022-01-27  73800  74000  71300  71300  2700
2022-01-28  71300  73700  71200  73300  2500
2022-04-29  65100  67600  65000  67400  2600
2022-07-07  56400  58700  56300  58200  2400


## 6. 종합 실습: 주가 데이터 분석 리포트

배운 내용을 종합하여 Apple 주가 분석 리포트를 작성합니다.

### 분석 항목
1. 연도별 평균 종가 비교
2. 월별 거래량 패턴
3. 최고가/최저가 기록일
4. 분석 결과 파일로 저장


In [15]:
# 종합 실습: 삼성전자 주가 분석 리포트
print("=" * 60)
print("[삼성전자 주가 분석 리포트]")
print("=" * 60)

# 기본 정보
print(f"\n분석 기간: {df.index[0].strftime('%Y-%m-%d')} ~ {df.index[-1].strftime('%Y-%m-%d')}")
print(f"총 거래일: {len(df)}일")

# 1. 연도별 평균 종가
print("\n" + "-" * 60)
print("[1. 연도별 평균 종가]")
yearly_avg = df.groupby(df.index.year)['Close'].mean()
for year, avg in yearly_avg.items():
    print(f"  {year}년: {avg:,.0f}원")

# 2. 전체 기간 통계
print("\n" + "-" * 60)
print("[2. 전체 기간 통계]")
print(f"  평균 종가: {df['Close'].mean():,.0f}원")
print(f"  최고 종가: {df['Close'].max():,}원")
print(f"  최저 종가: {df['Close'].min():,}원")
print(f"  평균 거래량: {df['Volume'].mean():,.0f}주")

[삼성전자 주가 분석 리포트]

분석 기간: 2022-01-03 ~ 2024-12-30
총 거래일: 735일

------------------------------------------------------------
[1. 연도별 평균 종가]
  2022년: 63,935원
  2023년: 67,457원
  2024년: 71,610원

------------------------------------------------------------
[2. 전체 기간 통계]
  평균 종가: 67,657원
  최고 종가: 87,800원
  최저 종가: 49,900원
  평균 거래량: 16,590,844주


In [16]:
# 3. 최고가/최저가 기록일
print("[3. 최고가/최저가 기록일]")
print("-" * 60)

# 최고 종가 기록일
max_close_date = df['Close'].idxmax()
max_close_value = df.loc[max_close_date, 'Close']
print(f"  최고 종가 기록일: {max_close_date.strftime('%Y-%m-%d')}")
print(f"  종가: {max_close_value:,}원")

# 최저 종가 기록일
min_close_date = df['Close'].idxmin()
min_close_value = df.loc[min_close_date, 'Close']
print(f"\n  최저 종가 기록일: {min_close_date.strftime('%Y-%m-%d')}")
print(f"  종가: {min_close_value:,}원")

# 최대 거래량 기록일
max_vol_date = df['Volume'].idxmax()
max_vol_value = df.loc[max_vol_date, 'Volume']
print(f"\n  최대 거래량 기록일: {max_vol_date.strftime('%Y-%m-%d')}")
print(f"  거래량: {max_vol_value:,}주")

[3. 최고가/최저가 기록일]
------------------------------------------------------------
  최고 종가 기록일: 2024-07-09
  종가: 87,800원

  최저 종가 기록일: 2024-11-14
  종가: 49,900원

  최대 거래량 기록일: 2024-01-11
  거래량: 57,691,266주


In [17]:
# 4. 분석 결과 저장
print("[4. 분석 결과 저장]")
print("-" * 60)

# 월별 요약 데이터 생성
monthly_summary = df.groupby(df.index.to_period('M')).agg({
        'Open': 'first',   # 월의 첫 거래일 시가
        'High': 'max',     # 월 중 최고가
        'Low': 'min',      # 월 중 최저가
        'Close': 'last',   # 월의 마지막 거래일 종가
        'Volume': 'sum'    # 월 전체 거래량 합계
})

# PeriodIndex를 문자열(YYYY-MM) 형태로 변환
monthly_summary.index = monthly_summary.index.astype(str)

# 컬럼명을 한글로 변경
monthly_summary.columns = ['Open', '최고가', '최저가', 'Close', '총거래량']

print("월별 요약 데이터 (최근 6개월):")
print(monthly_summary.tail(6))

# CSV로 저장
monthly_summary.to_csv('삼성전자_monthly_summary.csv')
print("\n저장 완료: 삼성전자_monthly_summary.csv")

# Excel로 저장 (여러 시트)
with pd.ExcelWriter('삼성전자_analysis_report.xlsx') as writer:
    df.to_excel(writer, sheet_name='일별데이터')
    monthly_summary.to_excel(writer, sheet_name='월별요약')
    yearly_avg.to_frame('평균종가').to_excel(writer, sheet_name='연도별평균')

print("저장 완료: 삼성전자_analysis_report.xlsx")
print("=" * 60)

[4. 분석 결과 저장]
------------------------------------------------------------
월별 요약 데이터 (최근 6개월):
          Open    최고가    최저가  Close       총거래량
Date                                          
2024-07  81500  88800  80000  83900  477159149
2024-08  86000  86100  70200  74300  402010176
2024-09  74500  74700  61500  61500  516679519
2024-10  60500  61900  55700  59200  537302984
2024-11  59000  59600  49900  54200  581402427
2024-12  54300  56600  51900  53200  387381948

저장 완료: 삼성전자_monthly_summary.csv
저장 완료: 삼성전자_analysis_report.xlsx


## 배운 내용 정리

### 1. FinanceDataReader
- `fdr.DataReader()`: 한국 주식 데이터를 무료로 가져오기
- OHLCV (시가, 고가, 저가, 종가, 거래량) 데이터 구조 이해

### 2. CSV 파일
- `df.to_csv()`: DataFrame을 CSV로 저장
- `pd.read_csv()`: CSV를 DataFrame으로 읽기
- 옵션: `index_col`, `parse_dates`, `encoding`

### 3. Excel 파일
- `df.to_excel()`: DataFrame을 Excel로 저장
- `pd.read_excel()`: Excel을 DataFrame으로 읽기
- `ExcelWriter`: 여러 시트에 데이터 저장

### 4. 날짜/기간 선택
- `df.loc['날짜']`: 특정 날짜 선택
- `df.loc['시작':'종료']`: 기간 슬라이싱
- `df.index.year`, `df.index.month`: 연도/월 필터링

### 5. 조건 필터링
- `df[조건]`: 조건에 맞는 행 선택
- `&` (AND), `|` (OR): 복합 조건

---

## 다음 차시 예고
다음 차시에서는 **데이터 시각화 (Matplotlib, Plotly)**를 배웁니다.
- 주가 차트 그리기
- 캔들스틱 차트
- 이동평균선 시각화
